In [ ]:
import os
import numpy as np
from numba import jit, cuda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.regularizers import l1
from sklearn.utils import shuffle
from PIL import Image
from tensorflow.keras import regularizers

In [ ]:
x_train = []
y_train = []
for i in os.listdir("../input/ethnicityrecog/UTKFace"):
#for i in os.listdir("../input/utk-face-cropped/utkcropped"):
    if(len(i.split('_')) == 4):
        age, gender, race, _ = i.split('_')
        age = int(age)
        gender = int(gender)
        race = int(race)
        if(age >= 20 and age <= 50):
            im =  Image.open("../input/ethnicityrecog/UTKFace/" + i)
            #im =  Image.open("../input/utk-face-cropped/utkcropped/" + i)
            im = im.resize((200,200),Image.ANTIALIAS)
            im = np.array(im)
            x_train.append(im)
            y_train.append(race)

In [ ]:
print(len(x_train))

In [ ]:
x = np.array(x_train)
y = to_categorical(y_train)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.90, random_state = 42)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size = 3, activation = 'relu', input_shape = (200,200,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))

#model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(32,activation = 'relu',kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(0.1))
model.add(Dense(5, activation = 'softmax',kernel_regularizer=regularizers.l2(0.01)))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs = 50,batch_size = 64)